In [47]:
import os
import sys
while not os.getcwd().endswith('ml'):
    os.chdir('..')
sys.path.insert(0, os.getcwd())

In [48]:
import math
import copy
import cv2
import numpy as np
import random
from shutil import copyfile
from pathlib import Path
from matplotlib import pyplot as plt
from kaggle_problems.rosneft_proppant import helpers
from kaggle_problems.rosneft_proppant.helpers import *
helpers.circleContour = ColoredCircleContour()

In [55]:
DATA_DIR = "kaggle_problems/rosneft_proppant/data"
ORIGINAL_IMG_DIR = "kaggle_problems/rosneft_proppant/data/train"
DEBUG_IMG_DIR = "kaggle_problems/rosneft_proppant/data/debug"
PROCESSING_IMG_DIR = "kaggle_problems/rosneft_proppant/data/processing"

INNER_SHAPE = (143.5, 86.5)
OUTER_SHAPE = (147.5, 90.5)
TARGET_SHAPE = (round(INNER_SHAPE[0] * 30), round(INNER_SHAPE[1] * 30))

In [50]:
def increase_brightness(img, value):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img


In [51]:
def check_size_contour(img, contour, down, up):
    area = cv2.contourArea(contour)
    return area > 1000 and \
        area / (img.shape[0] * img.shape[1]) >= down  and \
        area / (img.shape[0] * img.shape[1]) <= up

def is_rectange_area(contour):
    epsilon = 0.01 * cv2.arcLength(contour,True)
    res = cv2.approxPolyDP(contour,epsilon,True)
    return len(res) == 4 

def is_contours_simular(a, b, eps=0.1):
    return (abs(cv2.contourArea(a) - cv2.contourArea(b)) / (cv2.contourArea(a) + cv2.contourArea(b)) < eps) and \
        (abs(cv2.arcLength(a, True) - cv2.arcLength(b, True)) / (cv2.arcLength(a, True) + cv2.arcLength(b, True)) < eps)

In [52]:
def get_main_contour(thresh, find_inner):
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    hierarchy = hierarchy[0]
    origin_contours = copy.deepcopy(contours)
    contours = [(contour, i) for contour, i in zip(contours, range(len(contours)))]

    
    # filters
    contours = [(contour, i) for (contour, i) in contours if check_size_contour(img, contour, 0.1, 0.5)]
    contours = [(contour, i) for (contour, i) in contours if is_rectange_area(contour)]
    
    if (find_inner):
        set_of_contours = set([i for c, i in contours])
        contours = [(contour, i) for (contour, i) in contours if hierarchy[i][-1] in set_of_contours] # stay only if parent is rectangle
        contours = [(contour, i) for (contour, i) in contours if is_contours_simular(contour, origin_contours[hierarchy[i][-1]])]
    
    if (len(contours) != 1):
        return None

    return contours[0][0]


In [53]:
def order_rectangle(rectangle):
    return rectangle

def squeeze(rectangle):
    pass

def outer2inner(contour, img):
    epsilon = 0.01 * cv2.arcLength(contour, True)
    res = cv2.approxPolyDP(contour, epsilon, True)
    return contour

In [60]:
all_img = [img for img in os.listdir(ORIGINAL_IMG_DIR) if img.endswith('.jpg') ]

for img_name in all_img:
    print(img_name)
    img_number, _ = img_name.split('.')
    debug_dir = "{}/{}".format(DEBUG_IMG_DIR, img_number)
    Path(debug_dir).mkdir(exist_ok=True, parents=True)
    
    img = cv2.imread("{}/{}".format(ORIGINAL_IMG_DIR, img_name)) 
    origin_img = copy.deepcopy(img)
    img = increase_brightness(img, 100)    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    
    cv2.imwrite("{}/thresh.jpg".format(debug_dir), thresh)
    cv2.imwrite("{}/img_with_brightness.jpg".format(debug_dir), img)
    
    is_inner = True
    contour = get_main_contour(thresh, find_inner=is_inner)
    if contour is None:
        is_inner = False
        contour = get_main_contour(thresh, find_inner=is_inner)
        print("find outer contour for img_name: {}", img_name)
        
    if contour is None:
        print("ERROR: {}", img_name)
        continue
        
    # draw
    img_with_main_contour = img
    epsilon = 0.01 * cv2.arcLength(contour,True)
    res = cv2.approxPolyDP(contour,epsilon,True)
    cv2.drawContours(img, [res], -1, get_random_color(), 10)
    cv2.imwrite("{}/img_with_main_contour.jpg".format(debug_dir), img_with_main_contour)
    
    # трансформация
    res = np.float32([[i[0][0], i[0][1]] for i in res])
    res = order_rectangle(res)
    
    l = round(dist(res[0], res[1]))
    h = round(dist(res[1], res[2]))
   
    needed_contour = np.float32([[0, 0], [0, l], [h, l], [h, 0]])
   
    M = cv2.getPerspectiveTransform(res,needed_contour)
    dst = cv2.warpPerspective(origin_img, M, (h, l))
    
    if dst.shape[0] < dst.shape[1]:
        dst = cv2.rotate(dst, cv2.ROTATE_90_CLOCKWISE)
        
    assert(dst.shape[0] >= dst.shape[1])
        
    if not is_inner:
        x_diff = round((OUTER_SHAPE[0] - INNER_SHAPE[0]) / OUTER_SHAPE[0] * dst.shape[0] / 2)
        y_diff = round((OUTER_SHAPE[1] - INNER_SHAPE[1]) / OUTER_SHAPE[1] * dst.shape[1] / 2)
        
        dst = dst[x_diff:-x_diff, y_diff:-y_diff]
    
    # изменение пропорций картинки
    dst = cv2.resize(dst, (TARGET_SHAPE[1], TARGET_SHAPE[0]))
    
        
    main_area_dir = "{}/main_area".format(PROCESSING_IMG_DIR)
    Path(main_area_dir).mkdir(exist_ok=True, parents=True)
    
    cv2.imwrite("{}/{}".format(main_area_dir, img_name), dst)

63.jpg
189.jpg
77.jpg
638.jpg
604.jpg
162.jpg
176.jpg
88.jpg
610.jpg
348.jpg
360.jpg
406.jpg
412.jpg
374.jpg
599.jpg
228.jpg
214.jpg
572.jpg
566.jpg
200.jpg
758.jpg
770.jpg
764.jpg
765.jpg
771.jpg
759.jpg
567.jpg
201.jpg
215.jpg
573.jpg
229.jpg
598.jpg
413.jpg
375.jpg
361.jpg
407.jpg
349.jpg
find outer contour for img_name: {} 349.jpg
177.jpg
611.jpg
89.jpg
605.jpg
163.jpg
639.jpg
188.jpg
76.jpg
find outer contour for img_name: {} 76.jpg
62.jpg
74.jpg
60.jpg
48.jpg
149.jpg
613.jpg
175.jpg
161.jpg
607.jpg
388.jpg
439.jpg
377.jpg
411.jpg
405.jpg
363.jpg
559.jpg
203.jpg
565.jpg
571.jpg
217.jpg
767.jpg
773.jpg
772.jpg
766.jpg
570.jpg
216.jpg
202.jpg
564.jpg
558.jpg
404.jpg
362.jpg
376.jpg
410.jpg
438.jpg
389.jpg
160.jpg
606.jpg
612.jpg
174.jpg
148.jpg
49.jpg
61.jpg
75.jpg
59.jpg
71.jpg
65.jpg
170.jpg
616.jpg
602.jpg
164.jpg
158.jpg
399.jpg
414.jpg
372.jpg
366.jpg
400.jpg
428.jpg
560.jpg
206.jpg
212.jpg
574.jpg
548.jpg
789.jpg
762.jpg
776.jpg
777.jpg
763.jpg
788.jpg
549.jpg
213.jpg
find out

In [ ]:
all_img = [img for img in os.listdir(PROCESSING_IMG_DIR + "/main_area") if img.endswith('.jpg') ]

shapes = []
for img_name in all_img:
    img_number, _ = img_name.split('.')
    img = cv2.imread("{}/{}".format(PROCESSING_IMG_DIR + "/main_area", img_name))
    shapes.append(img.shape)

In [ ]:
v = [min(i[1], i[0]) / max(i[0], i[1]) for i in shapes]
mi = [min(i[1], i[0]) for i in shapes]
ma = [max(i[1], i[0]) for i in shapes]

In [ ]:
plt.hist(v, bins=100)
plt.show()
plt.hist(mi, bins=100)
plt.show()
plt.hist(ma, bins=100)
plt.show()